In [ ]:
import ee
import pandas as pd
from tqdm import tqdm

# Initialize the Earth Engine API (authentication required)
# ee.Authenticate()
# ee.Initialize()

# List of cities with latitude and longitude
cities = [
    {"name": "Vienna", "latitude": 48.2082, "longitude": 16.3738},
    {"name": "Munich", "latitude": 48.1351, "longitude": 11.5820},
    {"name": "New York", "latitude": 40.7128, "longitude": -74.0060},
    {"name": "Dubai", "latitude": 25.276987, "longitude": 55.296249},
    {"name": "Kabul", "latitude": 34.555349, "longitude": 69.207486},
    {"name": "Wunstorf", "latitude": 52.4670, "longitude": 9.4330}            
]

# Function to get brightness for a given city and year
def get_brightness(latitude, longitude, year):
    # Load the VIIRS dataset for the specified year
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG").filterDate(start_date, end_date)

    # Get the mean image for the year
    mean_image = viirs.mean()

    # Create a point for the city location
    point = ee.Geometry.Point([longitude, latitude])

    # Sample the brightness value at the city location
    brightness = mean_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=500
    ).get("avg_rad")

    return brightness.getInfo()

# Extract brightness values for each city across multiple years
years = [2012, 2017, 2023]

# Create a DataFrame to store the results
results = []

for city in tqdm(cities, desc="Processing cities"):
    lat, lon = city['latitude'], city['longitude']
    city_data = {"City": city["name"]}
    for year in years:
        city_data[f'BI_{year}'] = get_brightness(lat, lon, year)
    results.append(city_data)

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate the brightness change per decade
results_df['BI_change_decade'] = ((results_df['BI_2023'] - results_df['BI_2012']) / 11) * 10

# Display the updated DataFrame
print(results_df[['City', 'BI_2012', 'BI_2017', 'BI_2023', 'BI_change_decade']])

In [ ]:
import ee
import pandas as pd
import time
import os

# Initialize the Earth Engine API (authentication required)
# ee.Authenticate()
# ee.Initialize()

# Load metadata file with station information
metadata_file = r'C:\Users\Administrator\GHCN_stations_with_Landsat_GHSL_BU_1975to2020_metadata.csv'
metadata_df = pd.read_csv(metadata_file)

# Set batch size for processing
batch_size = 50

# Output file
output_file = metadata_file.replace('.csv', '_BI.csv')

# Do not delete the output file, continue from where it stopped
processed_batches = 0  # Start from batch 453
if os.path.exists(output_file):
    print(f"Resuming from batch {processed_batches + 1}")

# Function to get brightness for a given station and year
def get_brightness(latitude, longitude, year):
    # Load the VIIRS dataset for the specified year
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG").filterDate(start_date, end_date)

    # Get the mean image for the year
    mean_image = viirs.mean()

    # Create a point for the station location
    point = ee.Geometry.Point([longitude, latitude])

    # Sample the brightness value at the station location
    brightness = mean_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=1000
    ).get("avg_rad")

    return brightness.getInfo()

# Extract brightness values for each station across multiple years
years = [2012, 2017, 2020, 2023]

total_batches = (len(metadata_df) // batch_size) + (1 if len(metadata_df) % batch_size != 0 else 0)

for batch_index in range(processed_batches, total_batches):
    print(f"Processing batch {batch_index + 1}/{total_batches}")
    start_idx = batch_index * batch_size
    end_idx = min(start_idx + batch_size, len(metadata_df))
    batch_df = metadata_df.iloc[start_idx:end_idx].copy()

    for index, row in batch_df.iterrows():
        lat, lon = row['Lat'], row['Lon']
        for year in years:
            try:
                brightness = get_brightness(lat, lon, year)
            except Exception as e:
                print(f"Error processing station {row['Station_Name']} for year {year}: {e}")
                brightness = None
            batch_df.at[index, f'BI_{year}'] = round(brightness, 2) if brightness is not None else brightness
        time.sleep(0.01)  # Slight delay to avoid overwhelming Earth Engine servers

    batch_df['BI_change_decade'] = batch_df.apply(
        lambda x: round(((x['BI_2023'] - x['BI_2012']) / 11) * 10, 2) if pd.notnull(x['BI_2012']) and pd.notnull(x['BI_2023']) else None,
        axis=1
    )

    # Write the updated batch DataFrame to the output CSV file
    mode = 'a' if batch_index > 0 or processed_batches > 0 else 'w'
    header = (batch_index == 0 and processed_batches == 0)
    batch_df.to_csv(output_file, mode=mode, header=header, index=False)

    # Update the main DataFrame with the batch results
    metadata_df.update(batch_df)

print("Batch processing complete.")

In [ ]:
import pandas as pd
import os
import numpy as np
from scipy.stats import linregress

# Load existing output file
output_file = r'C:\Users\Administrator\GHCN_stations_with_Landsat_GHSL_BU_1975to2020_metadata_BI.csv'
if not os.path.exists(output_file):
    raise FileNotFoundError(f"The file {output_file} does not exist.")

result_df = pd.read_csv(output_file)

# Remove duplicates based on 'ID'
result_df = result_df.drop_duplicates(subset='ID', keep='last')

# Perform linear least squares fit for each station ID
bi_years = [2012, 2017, 2020, 2023]

# Initialize new columns
result_df['BI_2020_lsq'] = np.nan
result_df['BI_trend_lsq'] = np.nan

for index, row in result_df.iterrows():
    bi_values = [row.get(f'BI_{year}', np.nan) for year in bi_years]
    if all(pd.notnull(bi_values)):
        slope, intercept, _, _, _ = linregress(bi_years, bi_values)
        result_df.at[index, 'BI_2020_lsq'] = round(intercept + slope * 2020, 2)
        result_df.at[index, 'BI_trend_lsq'] = round(slope, 2)

# Display the updated DataFrame
print(result_df.head())

# Optionally, write the updated DataFrame back to the CSV
result_df.to_csv(output_file, index=False)